# Stack Overflow Dataset
In this notebook, we'll begin by performing some preliminary work on a Stack Overflow dataset found from the Kaggle platform. [(Link to dataset on Kaggle)](https://www.kaggle.com/datasets/aishu200023/stackindex) 

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import pandas as pd

In [2]:
# Loading in the Stack Overflow (SO) dataset as df
df = pd.read_csv('../data/MLTollsStackOverflow.csv')

In [3]:
# Viewing the first few rows of the dataset
df.head()

,month,nltk,spacy,stanford-nlp,python,r,numpy,scipy,matlab,machine-learning,...,Plato,Sympy,Flair,stanford-nlp.1,pyqt,Nolearn,Lasagne,OCR,Apache-spark-mlib,azure-virtual-machine
0,09-Jan,0,0,0,631,8,6,2,19,8,...,0,1,0,0,5,0,0,5,0,0
1,09-Feb,1,0,0,633,9,7,3,27,4,...,0,0,0,0,5,0,0,11,0,0
2,09-Mar,0,0,0,766,4,4,2,24,3,...,0,0,0,0,7,0,0,2,0,0
3,09-Apr,0,0,0,768,12,6,3,32,10,...,0,0,0,0,11,0,0,5,0,0
4,09-May,1,0,0,1003,2,7,2,42,7,...,0,0,0,0,10,0,0,3,0,0


## Interacting with the Dataset
By default, this dataset manifests in what is referred to as a **wide format**. The first thing we will do is convert the DataFrame into what is referred to as the **long format**.

In [4]:
# Converting dataset from wide format to long format
df_long = pd.melt(df, id_vars = 'month', var_name = 'topic', value_name = 'questions')

In [5]:
# Viewing the data now in long format
df_long.head(10)

,month,topic,questions
0,09-Jan,nltk,0.0
1,09-Feb,nltk,1.0
2,09-Mar,nltk,0.0
3,09-Apr,nltk,0.0
4,09-May,nltk,1.0
5,09-Jun,nltk,0.0
6,09-Jul,nltk,1.0
7,09-Aug,nltk,2.0
8,09-Sep,nltk,0.0
9,09-Oct,nltk,2.0


In [6]:
# Changing the 